In [1]:
from urllib import *
import json
import regex as re
import PySimpleGUI as sg
import webbrowser
import sys
import pprint
from dataclasses import dataclass
import hyperlink
import requests
import time

In [18]:
layout = [[sg.Text('Search bar'), sg.InputText(size=(30,40)), sg.Button('Search', bind_return_key=True, key="_SEARCH_")],
          [sg.Combo(['Search', 'Spell check'], default_value='Search', key="_type_")],
          [sg.Button('Previous page', bind_return_key=True, key="_prev_"), sg.Button('Next page', bind_return_key=True, key="_next_")],
          [sg.Output(font = ["Arial", 16], size=(180,50), key = '_output_')]
          ]

# Create the window http://localhost:8888/notebooks/Documents/School%20Work/2021%20Yr%203%20Sem%202/CE%204034/info-retrieval/Webpage%20demo.ipynb#
window = sg.Window("Demo Search Engine", layout, size=(1440, 850))

# Create an event loop
while True:
    event, values = window.read()
    
####################################################################
    if event == sg.WIN_CLOSED:
        break
        
####################################################################        
    if event == '_SEARCH_':
        
        window['_output_'].update(value='')
        
        wt = "wt=json"
        
        #change format of input to string
        string = str(values[0])
        
        #split terms in string into an array
        x = string.split()
        #print(x)
        
        lay = []
        say = ""
        
        #transform for query
        #no need if only one term or no term
        if (len(x)>1):
            for i in range(len(x)):
                say = say + x[i]
                if (i<(len(x)-1)):
                    say = say + " OR "
            say = "("+say+")"
            
        else:
            say = x[0]

        start = 0
            
        if(values['_type_']=='Spell check'):
    
            connection = requests.get('http://localhost:8983/solr/comment/spell', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
        #print(connection.json())
            response=connection.json()
            
            print("Query time: "+str(response['responseHeader']['QTime'])+" ms")
    
            #print(num)
        
            #print(response['spellcheck']['suggestions'][1]['suggestion'])
            for i in response['spellcheck']['suggestions'][1]['suggestion']:
                print(i['word'], " ", i['freq'])
        
        
        else:
            connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
            #print(connection.json())
            response=connection.json()
            
            print("Query time: "+str(response['responseHeader']['QTime'])+" ms")

            num = response['response']['numFound']
            #print(num)

            page_num = 0

            page_num = int((num+10-1)/10)

            #print(page_num)

            for i in range(len(response['response']['docs'])):
                lay.append(response['response']['docs'][i])

            records = 0
            for yiz in lay:
                records+=1
                print("Restaurant: "+yiz['restaurant'])
                if(len(yiz['type'])>1):
                    print("Type: "+yiz['type'][0]+", "+(yiz['type'][1].replace(" ", "")))
                else:
                    print("Type: "+yiz['type'][0])
                print("Reviewer: "+yiz['reviewer'][0])
                print("Rating: "+str(yiz['rating']))
                print("Comments: "+yiz['comment'])
                print("Pred_label: "+yiz['pred_label'])
                print("Score: "+str(yiz['score']))
                print("--------------------------------------------------")
                
            if(num%10==0):
                total = num//11+1
            else:
                total = num//10+1

            print("Page: "+str(start//10+1), "Total pages: "+str(total))
            print("Number of entries in this page: "+str(records)+" of "+str(num))
        

####################################################################
    if event == '_next_':
        
        start += 10
#         print("Num of response "+str(num))
#         print(start)
#         print(start//10+1)
        
        
        if(start>=num):
            print("You are at last page!!!")
            start=num//10*10-10
            continue
    
        window['_output_'].update(value='')
    
       
        lay = []
   
        
        #http://localhost:8983/solr/comment/select?q=asian
            
        connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
        #print(connection.json())
        response=connection.json()
        
        num = response['response']['numFound']
        
        for i in range(len(response['response']['docs'])):
                
            lay.append(response['response']['docs'][i])
            
        records=0
        
        for yiz in lay:
            records+=1
            print("Restaurant: "+yiz['restaurant'])
            if(len(yiz['type'])>1):
                print("Type: "+yiz['type'][0]+", "+(yiz['type'][1].replace(" ", "")))
            else:
                print("Type: "+yiz['type'][0])
            print("Reviewer: "+yiz['reviewer'][0])
            print("Rating: "+str(yiz['rating']))
            print("Comments: "+yiz['comment'])
            print("Pred_label: "+yiz['pred_label'])
            print("Score: "+str(yiz['score']))
            print("--------------------------------------------------")
            
            if(num%10==0):
                total = num//11+1
            else:
                total = num//10+1
                
        print("Page: "+str(start//10+1), "Total pages: "+str(total))
        print("Number of entries in this page: "+str(records)+" of "+str(num))
        
####################################################################         
    if event == '_prev_':
        
        start -= 10
#         print("Num of response "+str(num))
#         print(start)
#         print(start//10+1)
        
        if(start<0):
            print("You are at first page!!!")
            start=0
            continue
            
        window['_output_'].update(value='')
       
        lay = []
    
        #http://localhost:8983/solr/comment/select?q=asian
            
        connection = requests.get('http://localhost:8983/solr/comment/select', params={"q": say, 'wt' : 'json', 'rows' : 10, 'start' : start, 'indent' : True})
        #print(connection.json())
        response=connection.json()
        
        num = response['response']['numFound']
        
 
        for i in range(len(response['response']['docs'])):
                
                    
            lay.append(response['response']['docs'][i])
                    
        records=0
        
        for yiz in lay:
            records+=1
            print("Restaurant: "+yiz['restaurant'])
            if(len(yiz['type'])>1):
                print("Type: "+yiz['type'][0]+", "+(yiz['type'][1].replace(" ", "")))
            else:
                print("Type: "+yiz['type'][0])
            print("Reviewer: "+yiz['reviewer'][0])
            print("Rating: "+str(yiz['rating']))
            print("Comments: "+yiz['comment'])
            print("Pred_label: "+yiz['pred_label'])
            print("Score: "+str(yiz['score']))
            print("--------------------------------------------------")
            
            if(num%10==0):
                total = num//11+1
            else:
                total = num//10+1
                
        print("Page: "+str(start//10+1), "Total pages: "+str(total))
        print("Number of entries in this page: "+str(records)+" of "+str(num))
        
#################################################################### 

        
window.close()